In [1]:
import ipynb
from ipynb.fs.full.A_libs_26sep import *

In [2]:
def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [14]:
def create_output_dir(display = True):
    currentNotebook = notebook_path().split('/')
    currentNotebook = currentNotebook[-1]
    dir_models = '_'+currentNotebook[:-6]
    dir_models = dir_models.replace('.','_')
    output_dir = Path("../results/" + dir_models) #settings["globals"]["output_dir"])

    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
        if display:
            print("Directory " , output_dir ,  " Created ")
    elif display:
        print("Directory " , output_dir ,  " already exists")
    return output_dir

In [ ]:
# def log_loss_metric(y_true, y_pred):
#     y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
#     loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
#     return loss

def log_loss_score(actual, predicted,  eps=1e-15):

        """
        :param predicted:   The predicted probabilities as floats between 0-1
        :param actual:      The binary labels. Either 0 or 1.
        :param eps:         Log(0) is equal to infinity, so we need to offset our predicted values slightly by eps from 0 or 1
        :return:            The logarithmic loss between between the predicted probability assigned to the possible outcomes for item i, and the actual outcome.
        """
        
        p1 = actual * np.log(predicted+eps)
        p0 = (1-actual) * np.log(1-predicted+eps)
        loss = p0 + p1

        return -loss.mean()
    
def log_loss_multi(y_true, y_pred):
    M = y_true.shape[1]
    results = np.zeros(M)
    for i in range(M):
        results[i] = log_loss_score(y_true[:,i], y_pred[:,i])
    return results.mean()

# y_true = train_targets_scored[target_cols].values
# y_pred = valid_results[target_cols].values
# score = log_loss_metric(y_true, y_pred)
# score2 = log_loss_multi(y_true, y_pred)
# score, score2
# #(0.014310104262673127, 0.014310104262671468)